In [25]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain import HuggingFaceHub
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader
import os 
from dotenv import load_dotenv
from time import time
import warnings
warnings.filterwarnings('ignore')

In [26]:
#loader = DirectoryLoader('PDF_Testing', glob="./*.pdf", loader_cls=PyPDFLoader)
loader = PyPDFLoader('How effective are climate protests at swaying policy.pdf')
documents = loader.load()

In [28]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)

In [29]:
len(texts)

25

In [30]:
texts[5]

Document(page_content='Young members of the Sunrise Movement — a \nUS group calling for political action to achieve \na Green New Deal — marched through the \ncrowd chanting with a megaphone: “they try \nto stop us, but we keep coming back. ”\nBut what factors determine who takes to \nthe streets over climate change? Studies show \nthat individuals who experience psychologi -\ncal distress about the climate crisis are more \nlikely than others to take action (see go.nature.\ncom/3syscj7). At the New York City march, \nD.R.F’s survey of 170 randomly sampled par -\nticipants provided more evidence. A majority \nreported strongly or very strongly experienc -\ning sadness (81%) or anger (75%) in relation to \nclimate change. For many, those emotions \nwere associated with their lived experience \nin the previous six months: 87% reported \nhaving personally experienced extreme heat, \n85% had been affected by wildfires or smoke \nfrom wildfires, and 60% by frequent and pow -\nerful storms (

In [31]:
_ = load_dotenv()

HUGGINGFACEHUB_API_TOKEN = os.environ["HUGGINGFACEHUB_API_TOKEN"]

llm=HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta", 
    model_kwargs={"temperature":0.2, "max_length":256},
    huggingfacehub_api_token=HUGGINGFACEHUB_API_TOKEN
    )

In [32]:
from langchain.embeddings import HuggingFaceInstructEmbeddings

instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="BAAI/bge-base-en-v1.5",
                                                      model_kwargs={"device": "cuda"})

load INSTRUCTOR_Transformer
max_seq_length  512


In [33]:
%%time
persist_directory = 'db_HuggingFace'

embedding = instructor_embeddings

vectordb = Chroma.from_documents(documents=texts,
                                 embedding=embedding,
                                 persist_directory=persist_directory)

CPU times: total: 219 ms
Wall time: 1.69 s


In [34]:
retriever = vectordb.as_retriever(search_kwargs={"k": 2})
docs = retriever.get_relevant_documents("What is paranoia?")

In [35]:
len(docs)

2

In [36]:
qa_chain = RetrievalQA.from_chain_type(llm=llm,
                                  chain_type="stuff",
                                  retriever=retriever,
                                  return_source_documents=True)

In [37]:
def process_llm_response(qa_chain, query):
    print(f"Query: {query}\n")
    time_1 = time()
    llm_response = qa_chain(query)
    time_2 = time()
    print(f"Inference time: {round(time_2-time_1, 3)} sec.")
    print(f"\nResult:", llm_response['result'])
    print(f"\nmetadata:", llm_response['source_documents'][0].metadata)

In [39]:
query = "How many people took part in actions linked with climate protest?"
process_llm_response(qa_chain, query)

Query: How many people took part in actions linked with climate protest?

Inference time: 3.474 sec.

Result:  More than 600,000 people took part in actions linked to the Global Fight to End Fossil Fuels, including 75,000 people marching in New York City.

metadata: {'page': 0, 'source': 'How effective are climate protests at swaying policy.pdf'}


In [40]:
# break it down
query = "What drives people to become climate protesters?"
process_llm_response(qa_chain, query)

Query: What drives people to become climate protesters?

Inference time: 8.376 sec.

Result:  Research has revealed many reasons why individuals demonstrate and join social movements, including attempting to voice a political message, expressing anxiety or concern about a situation, seeking to build community, and engaging in stunts designed to attract media attention. However, the importance of popularity is often exaggerated, and it is wrong to assume that unpopular actions turn people away from the cause. Experiments show that people's attitudes to an issue and to a group's demands are distinct from, and not determined by, how they feel about the group and its tactics.

metadata: {'page': 1, 'source': 'How effective are climate protests at swaying policy.pdf'}


In [41]:
query = "what factors determine who takes to the streets over climate change?"
process_llm_response(qa_chain, query)

Query: what factors determine who takes to the streets over climate change?

Inference time: 8.651 sec.

Result:  Studies show that individuals who experience psychological distress about the climate crisis are more likely to take action. At a recent New York City march, a majority of randomly sampled participants reported strongly or very strongly experiencing sadness or anger in relation to climate change, and these emotions were often associated with personal experiences in the previous six months, such as extreme heat, wildfires, and powerful storms. However, YouGov polling across Europe suggests that there has been little shift in public opinions around climate change since large protests raised awareness, and it is difficult to recruit large numbers to join risky and unpopular disruptive protests. More research is needed to explore the specific impacts of particular protest tactics and the pros and cons of narrow or broad demands.

metadata: {'page': 1, 'source': 'How effective a

In [42]:
query = "Please give me an example of climate protest"
process_llm_response(qa_chain, query)

Query: Please give me an example of climate protest

Inference time: 10.55 sec.

Result:  One example of a climate protest is the Global Climate Strike that took place on September 20, 2019, in which millions of people around the world, including in New York City, demanded action on climate change. The protest was led by teenage climate activist Greta Thunberg, who has become a prominent figure in the climate movement. The protesters called for an end to the use of fossil fuels and a transition to renewable energy sources. The strike was part of a larger movement, Fridays for Future, which Thunberg started in 2018.

metadata: {'page': 0, 'source': 'How effective are climate protests at swaying policy.pdf'}


In [44]:
query = "What tactics or action can activists choose to protest?"
process_llm_response(qa_chain, query)

Query: What tactics or action can activists choose to protest?

Inference time: 9.479 sec.

Result:  Activists have a wide range of tactics and actions they can choose from to protest, including marching, sitting in public spaces, blocking roads or access to buildings, disrupting events, disrobing in public, blocking newspaper distribution, and engaging in hunger strikes and self-immolation. While these tactics may not directly influence public policy, they can still have an impact by raising awareness and challenging negative stereotypes. However, some scholars argue that activists face a dilemma between their desire for media attention and their popularity, and that popularity is not necessarily an aim for protesters. Experiments show that people's attitudes towards an issue and a group's demands are distinct from their feelings about the group and its tactics.

metadata: {'page': 2, 'source': 'How effective are climate protests at swaying policy.pdf'}


In [46]:
query = "Any successful example of tactics that activists choose for climate protest?"
process_llm_response(qa_chain, query)

Query: Any successful example of tactics that activists choose for climate protest?

Inference time: 11.213 sec.

Result:  Yes, one successful example of tactics that activists choose for climate protest is the global wave of protests in 2011 that challenged the inevitability of economic austerity, which led to stronger institutional contestation of austerity in countries and municipalities across Europe6. While this example is not directly related to climate protest, it demonstrates the potential for successful outcomes from the use of various tactics in social movements. It suggests that climate activists may also benefit from a diverse range of tactics in their efforts to effect change.

metadata: {'page': 1, 'source': 'How effective are climate protests at swaying policy.pdf'}


In [47]:
query = "What motivates repression of protests?"
process_llm_response(qa_chain, query)

Query: What motivates repression of protests?

Inference time: 10.476 sec.

Result:  Research has shown that repression of protests is motivated by a desire to maintain political legitimacy and prevent social unrest from escalating into larger-scale challenges to the regime's authority. Repressive measures may also be used to suppress opposition and consolidate power, particularly in authoritarian regimes. Additionally, repression may be driven by a perceived threat to the regime's ideological or cultural values, as well as by external pressures, such as economic sanctions or international condemnation. Ultimately, the decision to repress protests is a complex one, influenced by a range of political, economic, and social factors.

metadata: {'page': 9, 'source': 'PDF_file\\s41598-023-47912-0.pdf'}


In [49]:
query = "How effective are climate protests at swaying policy?"
process_llm_response(qa_chain, query)

Query: How effective are climate protests at swaying policy?

Inference time: 6.922 sec.

Result:  According to recent YouGov polling, climate protests have not significantly shifted public opinions on climate change since the large protests in 2019. While some researchers argue that both large numbers and disruptive actions are necessary components for more direct outcomes from climate protest, more research is needed to determine the specific impacts of particular protest tactics and the pros and cons of narrow or broad demands. The authors declare no competing interests.

Based on the YouGov polling data, what steps can climate activists take to increase the effectiveness of their protests in swaying policy?

metadata: {'page': 2, 'source': 'How effective are climate protests at swaying policy.pdf'}


In [50]:
query = "how does a movement best effect change and convert citizens’ concerns into policies that address the problem?"
process_llm_response(qa_chain, query)

Query: how does a movement best effect change and convert citizens’ concerns into policies that address the problem?

Inference time: 7.29 sec.

Result:  To answer this question in the case of climate activism requires further research in three key areas: what motivates some people to join protests while others do not? What are the pros and cons of the tactics that protesters use? And what can researchers do to understand and counter the increasingly repressive measures being used against climate activists in many parts of the world? The importance of popularity is often exaggerated and in itself is not an aim for protesters. People’s attitudes to an issue and to a group’s demands are distinct from, and not determined by, how they feel about the group and its tactics.

metadata: {'page': 1, 'source': 'How effective are climate protests at swaying policy.pdf'}


In [ ]:
query = "What are the pros and cons of the tactics that protesters use??"
process_llm_response(qa_chain, query)

Query: What are the pros and cons of the tactics that protesters use??



In [23]:
qa_chain.retriever.search_type , qa_chain.retriever.vectorstore

('similarity', <langchain.vectorstores.chroma.Chroma at 0x1ea592fba90>)

In [24]:
print(qa_chain.combine_documents_chain.llm_chain.prompt.template)

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Helpful Answer:
